In [1]:
from stog import models as Models

[2021-06-29 23:12:17,085 INFO] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
import torch
example = """
# ::id 1
# ::snt Bunga ditanam Bu Ani
(t / tanam
      :ARG0 (b1 / bu
            :name (a / ani))
      :ARG1 (b2 / bunga)))"""
example_path = "example_3.txt"
with open(example_path,"w") as f:
    f.write(example)

# Anotasi Fitur (POS, NER, LEM, TOK)
## Menggunakan CoreNLP Server 3.9.1

> Sebelum menjalankan script ini pastikan sudah ada CoreNLP server yang jalan di lokal dengan minimal 8GB RAM

> Kalau run pertama gagal, coba lagi, biasanya itu dikarenakan timeout. Karena disaat jalan awal dibutuhkan waktu lama untuk download package bahasa Inggris terlebih dahulu

Bila ingin menggunakan docker bisa dicoba docker registry berikut: [banditelol/CoreNLP](https://hub.docker.com/repository/docker/banditelol/corenlp) jangan lupa untuk membuka port yang dibutuhkan

## Compound File

Bisa di-download dari [repo ChunchuanLv](https://github.com/ChunchuanLv/AMR_AS_GRAPH_PREDICTION/blob/master/data/joints.txt). Ini isinya kata kerja untuk yang ada di propbank, kalau di Indonesia compound words verb seperti apa. 

Ini berisi compound word atau kata majemuk dalam bahasa Inggris. Untuk implementasi bahasa Indonesia berarti bisa untuk menangani masalah Rumah Sakit, Rumah Makan, etc.

In [5]:
from stog.data.dataset_readers.amr_parsing.io import AMRIO
for i, amr in enumerate(AMRIO.read("example_3.txt"), 1):
    print(amr)

# ::id 1
# ::snt Bunga ditanam Bu Ani
(t / tanam
      :ARG0 (b1 / bu
            :name (a / ani))
      :ARG1 (b2 / bunga))


In [7]:
!pip install stanza
import stanza

  Attempting uninstall: torch
    Found existing installation: torch 0.4.1
    Uninstalling torch-0.4.1:
      Successfully uninstalled torch-0.4.1


In [ ]:
nlp = stanza.Pipeline(lang="id",use_gpu=True,verbose=False, tokenize_pretokenized=True)

In [16]:
from stog.data.dataset_readers.amr_parsing.io import AMRIO
from stog.data.dataset_readers.amr_parsing.preprocess.feature_annotator import FeatureAnnotator
from stog.utils import logging
from tqdm.notebook import tqdm

logger = logging.init_logger()

compound_file= "data/AMR/amr_2.0_utils/joints.txt"
annotator = FeatureAnnotator('http://localhost:9000', compound_file)

logger.info('Processing {}'.format(example_path))
with open(example_path + '.features', 'w', encoding='utf-8') as f:
    for i, amr in enumerate(AMRIO.read(example_path), 1):
        if i % 1000 == 0:
            logger.info('{} processed.'.format(i))
        annotation = annotator(amr.sentence)
        amr.tokens = annotation['tokens']
        amr.lemmas = annotation['lemmas']
        amr.pos_tags = annotation['pos_tags']
        amr.ner_tags = annotation['ner_tags']
        AMRIO.dump([amr], f)
print(amr)

[2021-06-25 16:03:24,599 INFO] Processing example_2.txt


# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", 

# Preprocessing

Clean -> ~~Recategorize~~ -> Anonymization -> Sense Remover

What cleaning does:
```Python
    correct_errors(amr)
    normalize_tokens(amr)
    # Named entity
    join_model_name(amr)
    split_entity_with_slash(amr)
    split_entity_with_non(amr)
    split_entity_prefix(amr, 'anti')
    split_entity_prefix(amr, 'ex')
    split_entity_prefix(amr, 'cross')
    split_entity_prefix(amr, 'pro')
    replace_NT_dollar_abbr(amr)
    # Date
    join_time_description(amr)
    split_date_duration(amr)
    split_numerical_date(amr)
    split_year_month(amr)
    split_era(amr)
    split_911(amr)
    split_ratio(amr)
    split_unit_with_number(amr)
    split_number_with_dash_prefix(amr)
```

In [17]:
from stog.data.dataset_readers.amr_parsing.preprocess.input_cleaner import clean

util_dir = "data/AMR/amr_2.0_utils"
example_annotated = example_path + ".features"

with open(example_annotated + '.input_clean', 'w', encoding='utf-8') as f:
    for amr in AMRIO.read(example_annotated):
        clean(amr)
        f.write(str(amr) + '\n\n')
print(amr)

# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", 

#### Test data ga di Recategorize

Test data flownya lewat anonymization, ga recategorization

In [18]:
from stog.data.dataset_readers.amr_parsing.preprocess.recategorizer import Recategorizer

example_cleaned = example_annotated + ".input_clean"
amr_train_file = None
build_utils = False
dump_dir = util_dir

recategorizer = Recategorizer(
    train_data=amr_train_file,
    build_utils=build_utils,
    util_dir=dump_dir)

for amr in recategorizer.recategorize_file(example_cleaned):
    print(amr)


[2021-06-25 16:03:34,784 INFO] Done.



# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", "to", "wreck", "

### Test Data Anonymize

Untuk test data tidak perlu recategorize karena hanya perlu untuk mengambil informasi anonimisasi dari proses recategorize yang sudah dilakukan untuk training data. Karena pada dasarnya model ditrain pada vocabnya training data, jadi percuma kalau ini juga di recategorize.

In [19]:
from stog.data.dataset_readers.amr_parsing.preprocess.text_anonymizor import  TextAnonymizor
import os

text_anonymizor = TextAnonymizor.from_json(
    os.path.join(util_dir, "text_anonymization_rules.json"))

with open(example_cleaned + ".recategorize", "w", encoding="utf-8") as f:
    for amr in AMRIO.read(example_cleaned):
        print(amr)
        amr.abstract_map = text_anonymizor(amr)
        f.write(str(amr) + "\n\n")
        print(amr)

# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "Zhang", "Jizhong", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", 

Recat
{"NATIONALITY_1": {"type": "named-entity", "span": "Japanese", "ops": "Japan"}}
Anon
{"NATIONALITY_1": {"type": "named-entity", "span": "Japanese", "ner": "NATIONALITY", "ops": "Japan", "lemma": "Japan"}}

In [20]:
from stog.data.dataset_readers.amr_parsing.preprocess.sense_remover import SenseRemover
from stog.data.dataset_readers.amr_parsing.node_utils import NodeUtilities as NU

example_recategorized = example_cleaned + ".recategorize"    

node_utils = NU.from_json(util_dir, 0)

remover = SenseRemover(node_utils)


with open(example_recategorized + '.nosense', 'w', encoding='utf-8') as f:
    for amr in remover.remove_file(example_recategorized):
        f.write(str(amr) + '\n\n')
remover.reset_statistics()
print(amr)

# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", "to", "wreck", "

In [21]:
! mv example_2.txt.features.input_clean.recategorize.nosense example_2.txt.preproc

#  Prediction

``` bash
python -u -m stog.commands.predict \
    --archive-file ckpt-amr-2.0 \
    --weights-file ckpt-amr-2.0/best.th \
    --input-file data/AMR/amr_2.0/test.txt.features.preproc \
    --batch-size 16 \
    --use-dataset-reader \
    --cuda-device 0 \
    --output-file test.pred.txt \
    --silent \
    --beam-size 5 \
    --predictor STOG
```

In [22]:
from stog.commands.predict import _predict
from dataclasses import dataclass
import torch

@dataclass
class PredictArgs:
    archive_file: str = "ckpt-amr-2.0"
    weights_file: str = "ckpt-amr-2.0/best.th" 
    input_file: str = "example_2.txt.preproc"
    batch_size: int = 1
    use_dataset_reader: bool=True 
    cuda_device: int= 0 
    output_file: str= "example_2.pred.txt"
    silent: bool= False
    beam_size: int= 5 
    predictor: str= "STOG"

args = PredictArgs(silent = False   )
if args.cuda_device >= 0:
    device = torch.device('cuda:{}'.format(args.cuda_device))
else:
    device = torch.device('cpu')
args.cuda_device = device

# ini masalah karena ga batch, coba overwrite biar pun ga batch tetep bisa jalan buat predict
_predict(args)



[2021-06-25 16:03:35,477 INFO] loading archive file ckpt-amr-2.0
[2021-06-25 16:03:35,482 INFO] Loading token dictionary from ckpt-amr-2.0\vocabulary.
[2021-06-25 16:03:35,573 INFO] Building the STOG Model...
[2021-06-25 16:03:35,575 INFO] loading archive file data/bert-base-cased
[2021-06-25 16:03:35,589 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

[2021-06-25 16:03:51,742 INFO] encoder_token: 18002
[2021-06-25 16:03:51,744 INFO] encoder_chars: 113
[2021-06-25 16:03:51,746 INFO] decoder_token: 12202
[2021-06-25 16:03:51,747 INFO] decoder_chars: 87
[2021-06-25 16:04:04,374 INFO] loading vocabulary file data/bert-base-cased/bert-base-cased-vocab.txt
[2021-06-25 16:04:04,528 INFO] instantiating registered

process gather_index
[[0], [1], [2], [3], [4], [5], [6, 7, 8, 9, 10, 11], [12], [13], [14], [15], [16], [17, 18, 19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30, 31, 32], [33], [34], [35], [36], [37], [38], [39, 40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56]]
['[CLS]', 'not', 'like', 'the', 'old', 'guy', 'P', '##ER', '##SO', '##N', '##_', '##1', 'not', 'only', 'behave', 'like', 'a', 'ho', '##oli', '##gan', 'in', 'he', 'own', 'circle', ',', 'but', 'also', 'think', 'he', 'be', 'v', '##irt', '##uous', 'without', 'any', 'talent', '.', 'he', 'make', 't', '##v', 'series', 'when', 'he', 'have', 'nothing', 'to', 'do', 'to', 'wreck', 'people', 'of', 'the', 'whole', 'country', '.', '[SEP]']
token ids
[  101  1136  1176  1103  1385  2564   153  9637 23161  2249 28151  1475
  1136  1178 18492  1176   170 16358 11014  3820  1107  1119  1319  4726
   117  1133  1145  1341  1119  1129   191 25074  8163  1443  1251  5939
   119 

```
(m / multi-sentence
      :snt1 (c / contrast-01
            :ARG2 (g / guy
                  :ARG0-of (b / behave-01
                        :ARG1 (h / hooligan)
                        :location (c2 / circle
                              :poss g))
                  :ARG0-of (t / think-01
                        :ARG1 (v / virtuous
                              :domain g
                              :manner (t2 / talent
                                    :polarity -))
                        :mod (a2 / also))
                  :mod (o / old)
                  :name (n / name
                        :op1 "Zhang"
                        :op2 "Jizhong")
                  :wiki -))
      :snt2 (m2 / make-01
            :ARG0 (h3 / he)
            :ARG1 (s / series
                  :mod (t3 / television))
            :purpose (w / wreck-01
                  :ARG0 h3
                  :ARG1 (p / person
                        :poss (c3 / country
                              :mod (w2 / whole))))
            :time (h2 / have-03
                  :ARG0 g
                  :ARG1 (n2 / nothing
                        :ARG1-of (d / do-02)))))
```

```
(s / sit-01
      :ARG1 (a / and
            :op1 (s2 / sportsman)
            :op2 (p / person
                  :ARG0-of (c / coach-01))
            :op3 (d / doctor)
            :op4 (n / nurse)
            :op5 (p2 / person
                  :ARG0-of (h / have-org-role-91
                        :ARG1 (d2 / delegation)
                        :ARG2 (o2 / official))))
      :location (b / below
            :op1 (s3 / stage))
      :manner (n2 / neat))
```      


```
(h / hang
      :ARG1 (f / flag
            :mod (c2 / country
                  :name (n / name
                        :op1 "Japan")
                  :wiki "Japan"))
      :ARG2 (s / stage
            :poss (p / person
                  :ARG0-of (h2 / have-org-role
                        :ARG2 (c3 / chairman)))))
```

# Post Processing

In [23]:
from stog.data.dataset_readers.amr_parsing.postprocess.node_restore import NodeRestore
from stog.data.dataset_readers.amr_parsing.node_utils import NodeUtilities as NU
util_dir = "data/AMR/amr_2.0_utils" 
file_path = "example_2.pred.txt"

node_utils = NU.from_json(util_dir, 0)
nr = NodeRestore(node_utils)

with open(file_path + '.frame', 'w', encoding='utf-8') as f:
    for amr in nr.restore_file(file_path):
        print(amr)
        f.write(str(amr) + '\n\n')


# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", "to", "wreck", "

In [24]:
# from stog.data.dataset_readers.amr_parsing.postprocess.wikification import Wikification

# file_path = "example_2.pred.txt.frame"
# dump_spotlight_wiki = False

# wikification = Wikification(util_dir=util_dir)

# if dump_spotlight_wiki:
#     wikification.dump_spotlight_wiki(file_path)

# else:
#     wikification.load_utils()
#     with open(file_path + '.wiki', 'w', encoding='utf-8') as f:
#         for amr in wikification.wikify_file(file_path):
#             f.write(str(amr) + '\n\n')

In [25]:
from stog.data.dataset_readers.amr_parsing.postprocess.expander import Expander

file_path = "example_2.pred.txt.frame"

expander = Expander(util_dir=util_dir)

with open(file_path + '.expand', 'w', encoding='utf-8') as f:
    for amr in expander.expand_file(file_path):
        print(amr)
        f.write(str(amr) + '\n\n') 

[2021-06-25 16:04:09,269 INFO] Restored 1 name nodes.
[2021-06-25 16:04:09,270 INFO] Expanded 1 name nodes.
[2021-06-25 16:04:09,271 INFO] Expanded 0 date nodes.
[2021-06-25 16:04:09,271 INFO] Expanded 0 score nodes.
[2021-06-25 16:04:09,272 INFO] Expanded 0 ordinal nodes.
[2021-06-25 16:04:09,273 INFO] Expanded 0 quantities.
[2021-06-25 16:04:09,274 INFO] Expanded 0 urls.


# ::id bolt12_91455_5342.2 ::date 2012-12-11T05:38:25 ::annotator SDL-AMR-09 ::preferred
# ::snt Not like the old guy Zhang Jizhong not only behaved like a hooligan in his own circle, but also thinks he is virtuous without any talent. He makes TV series when he has nothing to do to wreck people of the whole country.
# ::tokens ["Not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behaved", "like", "a", "hooligan", "in", "his", "own", "circle", ",", "but", "also", "thinks", "he", "is", "virtuous", "without", "any", "talent", ".", "He", "makes", "TV", "series", "when", "he", "has", "nothing", "to", "do", "to", "wreck", "people", "of", "the", "whole", "country", "."]
# ::lemmas ["not", "like", "the", "old", "guy", "PERSON_1", "not", "only", "behave", "like", "a", "hooligan", "in", "he", "own", "circle", ",", "but", "also", "think", "he", "be", "virtuous", "without", "any", "talent", ".", "he", "make", "tv", "series", "when", "he", "have", "nothing", "to", "do", "to", "wreck", "

In [26]:
# ! sh ./tools/amr-evaluation-tool-enhanced/evaluation.sh example_2.pred.txt.frame.expand example_2.txt
! echo "Standard"
! C:\Python27\python.exe ./tools/amr-evaluation-tool-enhanced/smatch/smatch.py --pr -f example_2.pred.txt.frame.expand example_2.txt
# ! ls ./tools/amr-evaluation-tool-enhanced/
! echo "\nUnlabeled"
! C:\Python27\python.exe ./tools/amr-evaluation-tool-enhanced/unlabel.py "example_2.pred.txt.frame.expand"  > 1.tmp
! C:\Python27\python.exe ./tools/amr-evaluation-tool-enhanced/unlabel.py "example_2.txt"  > 2.tmp
! C:\Python27\python.exe ./tools/amr-evaluation-tool-enhanced/smatch/smatch.py --pr -f 1.tmp 2.tmp
# out=($out)
# echo 'Unlabeled -> P: '${out[1]}', R: '${out[3]}', F: '${out[6]} | sed 's/.$//'

# cat "$1" | perl -ne 's/(\/ [a-zA-Z0-9\-][a-zA-Z0-9\-]*)-[0-9][0-9]*/\1-01/g; print;' > 1.tmp
# cat "$2" | perl -ne 's/(\/ [a-zA-Z0-9\-][a-zA-Z0-9\-]*)-[0-9][0-9]*/\1-01/g; print;' > 2.tmp
# out=`python smatch/smatch.py --pr -f 1.tmp 2.tmp`
# out=($out)
# echo 'No WSD -> P: '${out[1]}', R: '${out[3]}', F: '${out[6]} | sed 's/.$//'

"Standard"
Precision: 0.548
Recall: 0.630
Document F-score: 0.586, 0.5862
"\nUnlabeled"
Precision: 0.590
Recall: 0.679
Document F-score: 0.632, 0.6316
